### MPLA Liste

In [2]:
## pip installs
import sys
!{sys.executable} -m pip install numpy
!{sys.executable} -m pip install pandas
!{sys.executable} -m pip install openpyxl



[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
import numpy as np
import pandas as pd


C:\Users\pcout\AppData\Local\Temp\ipykernel_16068\3269992828.py:2: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


## Comparative Analysis Checklist for MPLA Inland Dataset

### Preparation
- Load MPLA Inland input dataset and clean it.
- Load MPLA Output (Inland sheet) dataset and clean it.
- Identify and extract entries present in both input and output datasets (Included Entries).
- Identify and extract entries present in the input dataset but missing in the output dataset (Excluded Entries).

### Comparative Analysis Steps

#### Status Comparison
- Compare the frequency of each "Status" value between included and excluded entries.
- Identify any "Status" values predominantly associated with excluded entries.

#### Financial Figures Analysis
- Calculate and compare the average "Sollaufwand in €" between included and excluded entries.
- Calculate and compare the average "Kosten bisher berechnet (EUR)" between included and excluded entries.
- Assess if there's a financial threshold influencing inclusion or exclusion.

#### Date Field Examination
- Analyze the distribution of "Ursprüngliches Lieferdatum" for both groups.
- Analyze the distribution of "Abgestimmtes Lieferdatum" for both groups.
- Determine if specific date ranges or missing dates are common among excluded entries.

#### Information Completeness
- Check for the presence or absence of critical data across all relevant fields for both groups.
- Identify any fields where missing information correlates with exclusion.

### Analysis and Interpretation
- Interpret the data to identify significant differences between included and excluded entries.
- Look for patterns or criteria that could explain the reasons for exclusion.

### Additional Steps (if applicable)
- Refine the analysis based on initial findings to explore any uncovered patterns in greater depth.
- Apply identified criteria to a new dataset to validate the exclusion rules.


# Findings

### Anfrage:

### Angebot erstellt:
    - AIS Nummer gefüllt , Status aber aus SOM 
        (NEIN)
### Realisierungsphase:
    - Daueraufträge?
    - Endlosläufer
    - Nummern Speicher? (verconfigen und später Datenbanken)
### Testphase

### Generelles
- Kommentar fehlen
- abg. Lieferdatum in Vergangenheit (Delta 5 Tage mit Abruf)
- suffix fällt weg sobald AIS 



# Claudia



### Anfrage:

Som DB - Phase 

----------------------
fehlende Organisation ADP  => organisiation_adp
----------------------

benutzerdefinierte - mit Organisator adp

### Angebot erstellt:
   

### Realisierungsphase:
    
### Testphase








--------------

Fehlende Firmen durch Umfirmierung
---------------

Falsche Daten , einzeln in die Aufträge

## Aufbereitungsschritte:

- Log nicht vorhanden hinzufügen
- 

## Nachpflege 

- grundsätzlich falsch 

in AIS und SOM DB Status der Aufträge

In [4]:
## Preperation

mpla_inland_path = '../../files/input/mpla_inland.xlsx' 
mpla_output_path = '../../files/output/MPLA_OUTPUT_231120.xlsx'

mpla_inland_df = pd.read_excel(mpla_inland_path)
mpla_output_inland_df = pd.read_excel(mpla_output_path, sheet_name=0)

In [23]:
phase_column_name = 'Phase'

def clean_input(df_input):
    df = df_input.copy()
    df_header = df[df.iloc[:,2].str.contains('Firma', na=False)].index[0]
    df_cleaned = df.iloc[df_header+1:].reset_index(drop=True)
    df_cleaned.columns = df.iloc[df_header].values

    df_cleaned = add_phase_column(df_cleaned)
    df_cleaned = delete_prefix(df_cleaned, "Status")
    

    return df_cleaned

def clean_output(df_input, phase_column_name='Phase'):
    df = df_input.copy()
  
    # Find the header row index based on the column that contains 'Firma'
    df_header = df[df.iloc[:, 2].str.contains('Firma', na=False)].index[0]
    
    # Reset the DataFrame starting from the row after the header row
    df_cleaned = df.iloc[df_header+1:].reset_index(drop=True)
    
    # Set the column names to the values found in the identified header row
    df_cleaned.columns = df.iloc[df_header].values
    
    # Apply the phase column addition and cleaning to df_cleaned
    df_cleaned = add_phase_column(df_cleaned, column=0)
    
    return df_cleaned



## delete a prefix from a string - Prefix isnt fix but a number or number with dots combined 
def delete_prefix(df, column_name):
    df[column_name] = df[column_name].str.replace(r'^\d+\.?\d*\s?', '', regex=True)
    return df




def add_phase_column(df, column=0):
    df.rename(columns={
        df.columns[column]: phase_column_name
    }, inplace=True)

    
    ### ffill the phase column
    df[phase_column_name] = df[phase_column_name].ffill()

    cols_except_phase = df.columns.difference([phase_column_name])
    
    # Add a check to explicitly identify rows where all columns except 'Phase' are NaN
    # and mark them for exclusion
    df['AllOtherNaN'] = df[cols_except_phase].isna().all(axis=1)
    
    # Drop rows where all columns except 'Phase' are NaN or the 'Phase' row itself if it's the only non-NaN
    df = df[(df['AllOtherNaN'] == False)]
    
    # Remove the auxiliary 'AllOtherNaN' column as it's no longer needed
    df = df.drop(columns=['AllOtherNaN'])
    return df 

In [24]:
input_df_cleaned = clean_input(mpla_inland_df)
output_df_cleaned = clean_output(mpla_output_inland_df)


In [25]:
def missing_field_by_column(input, output, compare_column='AIS-Nummer'):
    input_ais_numbers = set(input[compare_column].astype(str).unique())
    output_ais_numbers = set(output[compare_column].astype(str).unique())

    missing_in_output = input_ais_numbers - output_ais_numbers

    return missing_in_output

def show_input_missing_in_output(input, output, compare_column='AIS-Nummer'):
    input_prepared = input.dropna(subset=[compare_column])
    output_prepared = output.dropna(subset=[compare_column])
    missing_in_output = missing_field_by_column(input_prepared, output_prepared, compare_column)

    return input[input[compare_column].astype(str).isin(missing_in_output)]


show_input_missing_in_output(input_df_cleaned, output_df_cleaned)

,Phase,AIS-Nummer,Firma,Projekt,Thema,Akt. Bearbeiter,Status,Anfragesteller,Ursprüngliches<br/>Lieferdatum,Abgestimmtes<br/>Lieferdatum,Kommentar / Bemerkung,Sollaufwand in €,Kosten bisher<br/>berechnet (EUR)
26,Angebot erstellt,2021-YWE-00084; 2021-YWE-00085,adp Gauselmann GmbH,NaN,Austausch der Handscanner Logistik,Jens Knollmann/BEIT GmbH,Neu,Philipp Obermark,NaN,NaN,NaN,"0,00","0,00"
28,Angebot erstellt,2023-CPR-00137,adp Merkur GmbH,NaN,Unterstützung bei der Auswahl einer Softwarelö...,Michael Schlömp,Angebot genehmigen,Juliane Engelking,NaN,NaN,Noch kein Log vorhanden,"6.300,00","0,00"
29,Angebot erstellt,2022-SNE-00026,adp Merkur GmbH,NaN,Controlling der An-/Abwesenheitszeiten der adp...,Susanne Neumann/BEIT GmbH,Neu,Volker Witting,NaN,NaN,\n\nAnforderungsbeschreibung Beobachtungsrepor...,"3.240,00","0,00"
36,Realisierungsphase,2016-DKÖ-00031,adp Gauselmann GmbH,NaN,"Prozesskoordination für die Systeme ""adp Inter...",Wolfgang Friedl,Auftragsdurchführung,Barbara Müller,31.12.2017,30.12.2023,"das ist ein Dauerauftrag, der auf den 31.12.20...","10.800,00","0,00"
48,Realisierungsphase,2008-CPR-00045,Merkur Freizeit Leasing GmbH,NaN,Prozesskoordination PhoenixX (Miete),Liane Gossen,Auftragsdurchführung,Michael Klingel,31.12.2009,31.12.2023,Dauerauftrag\n,"0,00","0,00"
58,Realisierungsphase,2023-CPR-00119,adp Merkur GmbH,NaN,Retrieval of Ukrainian currency UAH from the N...,Rüdiger Wunderlich,Auftragsdurchführung,Sonja Scholz,NaN,24.11.2023,"Hi Mirko,\n\ndie Ukrainische Gesellschaft hat ...","4.950,00","0,00"


In [22]:
show_input_missing_in_output(output_df_cleaned, input_df_cleaned)

'''
Wo kommt die her? 
    2023 - CPR - 00140 
    2023-SNE-00019	

 Warum kommen die mit?
 2023-CPR 00131 
 00112
 00085
 00127
 '''

,Phase,AIS-Nummer,Firma,Projekt,Thema,Projektleiter BEIT,Status,Anforderer/\nProjektleiter Kunde,geplantes\nLieferdatum,abgestimmtes \nLieferdatum,Kommentar zum abgestimmten/geänderten Lieferdatum,Sollaufwand in €,Kosten bisher berechnet €
13,Angebot erstellt,2023-CPR-00140,adp Merkur GmbH,NaN,Begleitung der Einführung der SAC im Bereich d...,Lenard Dresing,Angebot genehmigen,Philipp Obermark,NaN,NaN,NaN,4410,NaN
16,Realisierungsphase,2023-CPR-00131,MERKUR Service GmbH,NaN,"MSdZ - IT Antrag ""KTA&OHD Frontend - Hinweis 1...",Julian Greimann,Zweite Genehmigung,Klaus Schweppe,NaN,NaN,NaN,12330,0
29,Realisierungsphase,2023-CPR-00085,MERKUR Service GmbH,NaN,Übertragung Zeitbuchungen der OHD-Techniker in...,Julian Greimann,Auftragsdurchführung,Matthias Strecker,2023-10-31 00:00:00,2023-11-30 00:00:00,Der Fachbereich hat noch nicht getestet. Das ...,3420,1710
34,Realisierungsphase,2023-CPR-00112,MERKUR Service GmbH,NaN,Untersuchung Parallelbetrieb KTA und OHD auf g...,Julian Greimann,Auftragsdurchführung,Klaus Schweppe,NaN,2023-11-30 00:00:00,NaN,4860,3515
35,Realisierungsphase,2023-CPR-00127,MERKUR SERVICE GmbH,NaN,adp Portal: Anpassung Schnittstelle Containeru...,Michael Schlömp,Auftragsdurchführung,Sören Pinke,NaN,2023-12-08 00:00:00,NaN,3240,630
39,Realisierungsphase,2023-SNE-00019,MERKUR Service GmbH,NaN,MSdZ: Handlungsfähigkeit Phase 2 - Prozessbera...,Bernd Tinscher,Auftragsdurchführung,Joachim Mohrmann,2023-09-30 00:00:00,2023-12-28 00:00:00,Das ist ein Budgetauftrag mit der Leistung Ber...,4950,2542.5


In [26]:
show_input_missing_in_output(input_df_cleaned, output_df_cleaned, compare_column='Status')  ### Problem mit Präfix 



,Phase,AIS-Nummer,Firma,Projekt,Thema,Akt. Bearbeiter,Status,Anfragesteller,Ursprüngliches<br/>Lieferdatum,Abgestimmtes<br/>Lieferdatum,Kommentar / Bemerkung,Sollaufwand in €,Kosten bisher<br/>berechnet (EUR)
40,Realisierungsphase,2023-SNE-00026,adp Merkur GmbH,NaN,Erweiterung Suchfunktion um Datumseingrenzung ...,Julian Scholz,Abnahmeprotokoll versenden,Barbara Müller,NaN,18.08.2023,Die Entwicklungen des Auftrags befinden sich (...,"3.240,00","0,00"
59,Realisierungsphase,2023-CPR-00138,adp Merkur GmbH,NaN,adp-Cockpit: Optimierung Geräteliste Prio I,Michael Schlömp,Auftragsbestätigung versenden,Barbara Müller,NaN,NaN,"Hallo Claudia,\n\nden Liefertermin stimmen wir...","9.000,00","0,00"
60,Realisierungsphase,2023-CPR-00132,adp Merkur GmbH,NaN,EDI Abwicklung zwischen adp und AVNET/EBV,Nick Leon Siebrasse,Auftragsbestätigung versenden,Sven Schumacher,NaN,26.01.2024,Angebot liegt zur Genehmigung beim Geschäftsfü...,"2.175,00","0,00"


In [9]:
### load mpla ausland and mpla_output sheet 2 
mpla_ausland_path = '../../files/input/mpla_ausland.xlsx'
mpla_output_ausland_path = '../../files/output/MPLA_OUTPUT_231120.xlsx'

mpla_ausland_df = pd.read_excel(mpla_ausland_path)
mpla_output_ausland_df = pd.read_excel(mpla_output_ausland_path, sheet_name=1)

input_ausland_cleaned = clean_input(mpla_ausland_df)
output_ausland_cleaned = clean_output(mpla_output_ausland_df)

show_input_missing_in_output(input_ausland_cleaned, output_ausland_cleaned)


,Phase,AIS-Nummer,Firma,Projekt,Thema,Akt. Bearbeiter,Status,Anfragesteller,Ursprüngliches<br/>Lieferdatum,Abgestimmtes<br/>Lieferdatum,Kommentar / Bemerkung,Sollaufwand in €
6,Angebot erstellt,2022-CPR-000115,Merkur Gaming Distribution S.R.L,NaN,Tax Payments for Merkur Gaming Romania CC0135 ...,Claudia Prahl/BEIT GmbH,Neu,Sonja Scholz,NaN,NaN,"\n\nVon: Hodde, Melissa <MHodde@Gauselmann.de>...","2.790,00"


In [10]:
# 101034 - Ausland 
# 101038 - Inland

mpla_feb_inland_path = '../../files/input/Export_13_2_2024__10_10_34.xls'
mpla_feb_ausland_path = '../../files/input/Export_13_2_2024__10_10_38.xls'

mpla_feb_output_path = '../../files/output/MPLA 05.02.2024.xlsx'


mpla_feb_inland_df = pd.read_excel(mpla_feb_inland_path)
mpla_feb_ausland_df = pd.read_excel(mpla_feb_ausland_path)
mpla_feb_inland_output_df = pd.read_excel(mpla_feb_output_path, sheet_name=0)
mpla_feb_ausland_output_df = pd.read_excel(mpla_feb_output_path, sheet_name=1)


c:\Users\pcout\AppData\Local\Programs\Python\Python312\Lib\site-packages\openpyxl\styles\stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
c:\Users\pcout\AppData\Local\Programs\Python\Python312\Lib\site-packages\openpyxl\styles\stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


In [11]:
show_input_missing_in_output(clean_input(mpla_feb_inland_df), clean_output(mpla_feb_inland_output_df))

,Phase,AIS-Nummer,Firma,Projekt,Thema,Akt. Bearbeiter,Status,Anfragesteller,Ursprüngliches<br/>Lieferdatum,Abgestimmtes<br/>Lieferdatum,Kommentar / Bemerkung,Sollaufwand in €
6,Angebot erstellt,2022-CPR-000115,Merkur Gaming Distribution S.R.L,NaN,Tax Payments for Merkur Gaming Romania CC0135 ...,Claudia Prahl/BEIT GmbH,Neu,Sonja Scholz,NaN,NaN,"\n\nVon: Hodde, Melissa <MHodde@Gauselmann.de>...","2.790,00"
7,Angebot erstellt,2023-SNE-00013,Merkur Dosniha S. L.,NaN,DHL Express Spain - paperless trade for parcel...,Klaus Hartlage,Angebot,amartinez,NaN,NaN,on hold\n,"6.470,00"
8,Angebot erstellt,2024-CPR-00019,Merkur Dosniha S. L.,NaN,it.x-press interface SEUR Spain,Klaus Hartlage,Angebot,jmayol,NaN,NaN,Noch kein Log vorhanden,"12.226,00"
10,Realisierungsphase,2022-CPR-00091,Merkur Dosniha S. L.,NaN,SII Support - MERKUR Dosniha 2022/23 Part I,Sebastian Hofbüker,Auftragsdurchführung,jmayol,NaN,30.12.2023,Abstimmung: Budgetverlängerung\n,"10.575,00"
11,Realisierungsphase,2022-CPR-00102,Merkur Gaming Poland Sp z.o.o.,NaN,TIS for MERKUR Gaming Poland cc 0102,Jens Knollmann,Auftragsdurchführung,Sonja Scholz,15.03.2023,29.02.2024,Anpassung LT = 29.02.2024 (Basis: Mail von M. ...,"3.780,00"
12,Realisierungsphase,2023-CPR-00029,Merkur Gaming Slots d.o.o. Beograd,NaN,TIS for MERKUR Gaming Slots (0138) - RS - Uni...,Jens Knollmann,Auftragsdurchführung,Valerie Redcher,16.06.2023,30.04.2024,Anpassung LT = 30.04.2024 (Basis: Mail von M. ...,"7.110,00"
13,Realisierungsphase,2023-CPR-00156,Blueprint Operations,NaN,Material numbers for refurbished components at...,Catrinel Gavrilescu,Auftragsbestätigung,Nadia Volosina,NaN,31.03.2024,Noch kein Log vorhanden,"11.400,00"
14,Realisierungsphase,2024-CPR-00010,Merkur Dosniha S. L.,NaN,New Liferay-Entity for MERKUR Dosniha (PORTAL-...,Michael Schlömp,Auftragsdurchführung,jmayol,NaN,15.03.2024,Noch kein Log vorhanden,"2.400,00"
16,Testphase,2023-CPR-00113,Merkur Gaming Slots d.o.o. Beograd,NaN,SD Light Rollout at Merkur Gaming Slots,Julian Greimann,Kundentest,Sonja Scholz,30.11.2023,31.01.2024,"Der Auftrag wird bis zum 31.1.24 verlängert, d...","4.230,00"


In [12]:
show_input_missing_in_output(clean_input(mpla_feb_ausland_df), clean_output(mpla_feb_ausland_output_df))

,Phase,AIS-Nummer,Firma,Projekt,Thema,Akt. Bearbeiter,Status,Anfragesteller,Ursprüngliches<br/>Lieferdatum,Abgestimmtes<br/>Lieferdatum,Kommentar / Bemerkung,Sollaufwand in €,Kosten bisher<br/>berechnet (EUR)
6,Anfrage,2023-SNE-00046,adp Merkur GmbH,NaN,Konzeption Umstrukturierung BW-Rollen für adp ...,Ekbal Al Koppa,Erstellen,Jan-Eric Sanders,NaN,NaN,"Hallo zusammen,\n\n\n\ndieses Angebot wurde ge...","6.480,00","0,00"
13,Anfrage,2023-CPR-00078,adp Merkur GmbH,NaN,Systemtrennung der FirstSpirit Infrastruktursy...,Wolfgang Friedl,Erstellen,Barbara Müller,NaN,NaN,Log leer,"0,00","0,00"
33,Angebot erstellt,2021-YWE-00084; 2021-YWE-00085,adp Gauselmann GmbH,NaN,Austausch der Handscanner Logistik,Jens Knollmann/BEIT GmbH,Neu,Philipp Obermark,NaN,NaN,NaN,"0,00","0,00"
34,Angebot erstellt,2023-SNE-00042,adp Merkur GmbH,NaN,Ablageumgebung und Scanarbeitsplatz für SEPA M...,Jonas Rabe,Angebot,Jan Grefe,NaN,NaN,Fachbereich sieht Klärungsbedarf. Beit wird no...,"2.351,00","0,00"
35,Angebot erstellt,2022-SNE-00026,adp Merkur GmbH,NaN,Controlling der An-/Abwesenheitszeiten der adp...,Susanne Neumann/BEIT GmbH,Neu,Volker Witting,NaN,NaN,\n\nAnforderungsbeschreibung Beobachtungsrepor...,"3.240,00","0,00"
36,Angebot erstellt,2023-CPR-00007,adp Merkur GmbH,NaN,adp MERKUR: SAP BW Leistungen 2024,Claudia Prahl/BEIT GmbH,Neu,Jochen Clemens,NaN,NaN,NaN,"96.000,00","0,00"
37,Angebot erstellt,2024-CPR-00016,adp Merkur GmbH,NaN,Migration und Konsolidierung der Atlassian Pro...,Michael Schlömp,Angebot,Frank Gläser,NaN,NaN,Noch kein Log vorhanden,"83.340,00","0,00"
38,Angebot erstellt,2024-CPR-00018,adp Merkur GmbH,NaN,"it.x-press Schnittstellen core Upgrade, UPS Up...",Klaus Hartlage,Angebot,Philipp Obermark,NaN,NaN,Noch kein Log vorhanden,"16.800,00","0,00"
39,Angebot erstellt,2024-CPR-00015,adp Merkur GmbH,NaN,SAP Analytics Cloud Nutzung im GB Merkur,Lenard Dresing,Angebot,Sonja Scholz,NaN,NaN,Noch kein Log vorhanden,"5.705,00","0,00"
40,Angebot erstellt,2024-CPR-00025,adp Merkur GmbH,NaN,IT Strategie adp Merkur: Fortschreibung der Zi...,Jörg Sefzig,Angebot,Jochen Clemens,NaN,NaN,Noch kein Log vorhanden,"49.920,00","0,00"


############# Spielwiese #########